In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./Sentiment.csv')
data.head(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [3]:
data = data[['text','sentiment']]
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [4]:
data = data[data.sentiment != "Neutral"]
data['sentiment']= pd.get_dummies(data['sentiment'], drop_first = True)
data = data.reset_index(drop=True)
data.head()

,text,sentiment
0,RT @ScottWalker: Didn't catch the full #GOPdeb...,True
1,RT @RobGeorge: That Carly Fiorina is trending ...,True
2,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,True
3,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",True
4,RT @warriorwoman91: I liked her and was happy ...,False


In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [6]:
corpus = []

for i in range(0, data.shape[0]):
    tweet = re.sub('[^a-zA-Z]', ' ', data['text'][i])
    tweet = tweet.lower()
    tweet = word_tokenize(tweet)
    
    # Reduce words to their root form
    tweet = [WordNetLemmatizer().lemmatize(w) for w in tweet if not w in set(stopwords.words('english'))]
    
    # Lemmatize verbs by specifying pos
    tweet = [WordNetLemmatizer().lemmatize(w, pos='v') for w in tweet if not w in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

print(corpus[0:3])

['rt scottwalker catch full gopdebate last night scott best line second walker http co zsff', 'rt robgeorge carly fiorina trend hour debate men complete gopdebate say', 'rt danscavino gopdebate w realdonaldtrump deliver highest rat history presidential debate trump http co']


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(corpus)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
pd.set_option('display.max_rows', None)
df.head(20)

,TF-IDF
zsff,0.453518
catch,0.344600
full,0.330405
second,0.313011
line,0.299403
scottwalker,0.282078
scott,0.274517
best,0.249607
walker,0.235920
last,0.182394


In [8]:
for i in range(0, len(corpus)):
    corpus[i] = re.sub('co','', corpus[i])
    corpus[i] = re.sub('rt','', corpus[i])
    corpus[i] = re.sub('http','', corpus[i])
    

In [ ]:
pip install tensorflow==2.8.0

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(corpus)
encoded_docs = tokenizer.texts_to_sequences(corpus)
padded_sequence = pad_sequences(encoded_docs,maxlen=25)

ImportError: Could not find the DLL(s) 'msvcp140_1.dll'. TensorFlow requires that these DLLs be installed in a directory that is named in your %PATH% environment variable. You may install these DLLs by downloading "Microsoft C++ Redistributable for Visual Studio 2015, 2017 and 2019" for your platform from this URL: https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads

In [15]:
pip list


Package                       Version
----------------------------- ---------------
absl-py                       2.1.0
alabaster                     0.7.12
anyio                         4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
astunparse                    1.6.3
async-lru                     2.0.4
atomicwrites                  1.4.0
attrs                         23.1.0
audioread                     3.0.1
Automat                       20.2.0
autopep8                      2.0.4
Babel                         2.11.0
backcall                      0.2.0
bcrypt                        3.2.0
beautifulsoup4                4.12.2
binaryornot                   0.4.4
black                         23.11.0
bleach                        4.1.0
bokeh                         3.1.1
Bottlene

In [10]:
padded_sequence.shape

NameError: name 'padded_sequence' is not defined

In [20]:
print(tokenizer.word_index['trump'])

7


In [21]:
print(corpus[0])
print(encoded_docs[0])

rt scottwalker catch full gopdebate last night scott best line second walker http co zsff
[2, 253, 782, 569, 1, 18, 16, 207, 117, 355, 457, 74, 4, 5, 5300]


In [22]:
print(padded_sequence[0])

[   0    0    0    0    0    0    0    0    0    0    2  253  782  569
    1   18   16  207  117  355  457   74    4    5 5300]


In [20]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from keras.initializers import Constant

vocab_size = len(tokenizer.word_index) + 1
embedding_vector_length = 200


model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,
                    input_length=25) )
model.add(SpatialDropout1D(0.2))
model.add(LSTM(3, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 200)           2598000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 25, 200)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 3)                 2448      
                                                                 
 dropout (Dropout)           (None, 3)                 0         
                                                                 
 dense (Dense)               (None, 1)                 4         
                                                                 
Total params: 2,600,452
Trainable params: 2,600,452
Non-trainable params: 0
______________________________________________

In [21]:
# converting the targets to numpy array to feed it into the model
target = np.asarray(data['sentiment'])
print(target)

[1 1 1 ... 1 0 1]


In [22]:
MODEL = model.fit(padded_sequence,target,validation_split=0.2, epochs=5, batch_size=256)

Epoch 1/5
34/34 [==============================] - 12s 189ms/step - loss: 0.6471 - accuracy: 0.7563 - val_loss: 0.5776 - val_accuracy: 0.7968
Epoch 2/5
34/34 [==============================] - 5s 162ms/step - loss: 0.5404 - accuracy: 0.7939 - val_loss: 0.5031 - val_accuracy: 0.7968
Epoch 3/5
34/34 [==============================] - 6s 165ms/step - loss: 0.4775 - accuracy: 0.8024 - val_loss: 0.4809 - val_accuracy: 0.8010
Epoch 4/5
34/34 [==============================] - 5s 162ms/step - loss: 0.4262 - accuracy: 0.8332 - val_loss: 0.4539 - val_accuracy: 0.8080
Epoch 5/5
34/34 [==============================] - 6s 164ms/step - loss: 0.3718 - accuracy: 0.8681 - val_loss: 0.3997 - val_accuracy: 0.8169


In [24]:
test_word ='Pune is a not a good city'
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=25)
sentiment = int(model.predict(tw).round().item())
#print(sentiment)
if sentiment==0:
    print("Negative")
else:
    print("Positive")


Negative
